In [1]:
import numpy as np
import torch
from torch.nn import MSELoss
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import time
from torch import nn
import pickle
from sklearn.metrics import mean_squared_error as MSE
from collections import Counter
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools

In [3]:
test_biopsy_unbalenced = pd.read_csv("./final_splits/test_biopsy_unbalenced.csv")

In [4]:
preds_df_under_54 = pd.read_csv('preds_under_54.csv', index_col=0)
preds_between_54_55 = pd.read_csv('preds_between_54_55.csv', index_col=0)

In [5]:
preds_df_all = pd.DataFrame()
preds_df_all[ list(preds_df_under_54) ] = preds_df_under_54
preds_df_all[ list(preds_between_54_55)[2:] ] = preds_between_54_55.iloc[:,2:]

In [6]:
preds = preds_df_all.iloc[:,2:].values.T

In [16]:
pred_nr = np.arange(preds.shape[0])

for c in range(1,len(pred_nr)):
    
    c = list(itertools.combinations(pred_nr, c))

    subsets = np.array([list(i) for i in c])

    best_MSE = 1

    for s in tqdm(range(len(subsets))):

        current_MSE = MSE(np.mean(np.array(preds)[[subsets[s]]], axis=1)[0], test_biopsy_unbalenced.stage.values)

        if current_MSE < best_MSE:
            best_MSE = current_MSE
            best_s = s

    print(best_s, subsets[best_s], best_MSE)    

100%|██████████| 27/27 [00:00<00:00, 849.74it/s]


3 [3] 0.5369292398599026


100%|██████████| 351/351 [00:00<00:00, 2427.00it/s]


324 [19 21] 0.506007898485689


100%|██████████| 2925/2925 [00:01<00:00, 2350.71it/s]


...

163252 [ 3  4  6 10 14 21] 0.4914808400090951


100%|██████████| 888030/888030 [06:32<00:00, 2264.29it/s]


550701 [ 3  4  6 10 14 21 22] 0.4912734082561767


 57%|█████▋    | 1258534/2220075 [08:05<06:11, 2590.67it/s]


KeyboardInterrupt: 

In [19]:
MSE(test_biopsy_unbalenced.stage.values, np.mean(preds, axis=0))

0.5041816193969849

In [8]:
selected_models = np.array(list(preds_df_all)[2:])
selected_models = selected_models[[3,  4,  6, 10, 14, 21, 22]]

In [9]:
selected_models

array(['biopsy_model_4_bracs_3_0_53897.pth',
       'biopsy_model_4_bracs_0_5346.pth',
       'biopsy_model_4_bracs_1_0_53875.pth',
       'biopsy_model_4_bracs_2_0_55158.pth',
       'biopsy_model_4_bracs_1_0_54099.pth',
       'biopsy_model_4_bracs_1_0_54368.pth',
       'biopsy_model_4_bracs_3_0_55024.pth'], dtype='<U34')

### inference on holdout set

In [11]:
def inference(x, model, device):
    
    
    bag = np.load("/home/ngsci/project/save_resnet_embeddings_level4_biopsy_bags_bracs_float16/"+ x["biopsy_id"] + ".npy")
    # create a batch of 1
    if bag.shape[0] == 0: print('nulla')
    bag = torch.tensor(np.expand_dims(bag,axis=0))
    prediction = model(bag.float().to(device))
    
    return prediction.item()

In [12]:
def run_inference_on_model(model_name, csv_name="./final_splits/test_biopsy_unbalenced.csv"):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #device config
    
    model = torch.load(f"./trained_models/{model_name}", map_location=torch.device('cpu'))
    model = model.module.to(device)
    
    test_biopsy_unbalenced = pd.read_csv(csv_name)
    
    #test_biopsy_unbalenced["stage_pred"] = test_biopsy_unbalenced.apply(lambda x: inference(x, model, device),axis=1)
    
    return test_biopsy_unbalenced.apply(lambda x: inference(x, model, device),axis=1).values

In [13]:
preds_holdout = []

for m in tqdm(selected_models):
    
    preds_holdout.append(run_inference_on_model(m, csv_name="./final_splits/HOLD_OUT.csv"))

100%|██████████| 7/7 [05:30<00:00, 47.17s/it] 


In [14]:
pred_holdout_ensemble = pd.read_csv("./final_splits/HOLD_OUT.csv", index_col=0)
pred_holdout_ensemble['stage_pred'] =  np.mean(np.array(preds_holdout), axis=0)

filt_4higher = pred_holdout_ensemble['stage_pred'] >= 4.0
pred_holdout_ensemble["stage_pred"][filt_4higher] = 4.0

filt_0lower = pred_holdout_ensemble['stage_pred'] < 0.0
pred_holdout_ensemble["stage_pred"][filt_0lower] = 0.0

/tmp/ipykernel_297/3130930446.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_holdout_ensemble["stage_pred"][filt_4higher] = 4.0
/tmp/ipykernel_297/3130930446.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_holdout_ensemble["stage_pred"][filt_0lower] = 0.0


In [ ]:
pred_holdout_ensemble.to_csv(  f'./preds/pred_{len(preds_holdout)}_ensemble_comb1.csv', index=False)

In [ ]:
import ngsci

ngsci.submit_contest_entry(
    f'./preds/pred_{len(preds_holdout)}_ensemble_comb1.csv', description="csabAIbio ensemble comb1"
)